In [12]:
# Importando bibliotecas

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import train_test_split

%matplotlib inline

# (1, 28, 28) Theano
# (28,28,1) Tensorflow
# nc = 1 --> NUmber of channels


In [13]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [14]:
train.shape

(42000, 785)

In [15]:
train.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [16]:
train['label'].unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6], dtype=int64)

In [17]:
train.isnull().sum().any()

False

In [18]:
label_train = train['label']                               # get dependent variable 
train.drop(['label'],axis=1,inplace=True)              # drop it from original dataframe

In [19]:
train = train.values.reshape(-1,28,28,1).astype('float32')      # reshape to proper shape
test = test.values.reshape(-1,28,28,1).astype('float32')        # reshape to proper shape
print(train.shape)
print(test.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [20]:
train = train/255
test = test/255

In [21]:
label_train = np_utils.to_categorical(label_train)

In [22]:
num_classes = label_train.shape[1]
print(num_classes)

10


In [23]:
model = Sequential()
model.add(Conv2D(30,(5,5),input_shape=(28,28,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(num_classes, activation='softmax', name='prediction'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 30)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 30)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 30)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4320)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               553088    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)               

In [24]:
x_train, x_test, y_train, y_test = train_test_split(train, label_train,test_size=0.2,random_state=42)         # splitting data into train and test for validation later on

In [25]:
# datagen = ImageDataGenerator()

# train_datagen = datagen.flow(x_train,y_train,batch_size=50)                        # flow training data in batches of 50 to the model

In [26]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=10, batch_size=200)
# fit the model with training and validation generators

Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 10s 310us/step - loss: 0.4081 - accuracy: 0.8776 - val_loss: 0.1545 - val_accuracy: 0.9558
Epoch 2/10
33600/33600 [==============================] - 12s 349us/step - loss: 0.1122 - accuracy: 0.9657 - val_loss: 0.0965 - val_accuracy: 0.9729
Epoch 3/10
33600/33600 [==============================] - 12s 343us/step - loss: 0.0758 - accuracy: 0.9765 - val_loss: 0.0671 - val_accuracy: 0.9794
Epoch 4/10
33600/33600 [==============================] - 12s 343us/step - loss: 0.0562 - accuracy: 0.9826 - val_loss: 0.0599 - val_accuracy: 0.9817
Epoch 5/10
33600/33600 [==============================] - 12s 346us/step - loss: 0.0459 - accuracy: 0.9858 - val_loss: 0.0665 - val_accuracy: 0.9795
Epoch 6/10
33600/33600 [==============================] - 11s 339us/step - loss: 0.0374 - accuracy: 0.9879 - val_loss: 0.0576 - val_accuracy: 0.9810
Epoch 7/10
33600/33600 [==============================] -

In [27]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("\nAccuracy: %.2f%% " % (scores[1]*100))


Accuracy: 98.32% 


In [33]:
prediction = model.predict(test)                    # use model to predict for all test values

In [34]:
submission = np.argmax(prediction,axis=1)           # since results are in categorical form, choose the highest in each and store them
submission

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [35]:
my_submission = pd.DataFrame({'ImageId': range(1,len(test)+1) ,'Label':submission })               # make dataframe with the column headders and predicted values

my_submission.to_csv("results.csv",index=False)

In [36]:
my_submission.head(30)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3
